<a href="https://colab.research.google.com/github/bnf99/FiscaComputacional1/blob/main/Tarea1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea semana 2: Intergración y Derivación Numérica.
## IF4702 - Física computacional 1 

### Profesores:
> José Esteban Pérez Hidalgo  
> Álvaro Amador Jara

### Estudiante:
> Bryam Gerardo Núñez Flores (2018111986)

### Repositorio:
> [github.com/bnf99/FiscaComputacional1](https://github.com/bnf99/FiscaComputacional1)


In [12]:
import numpy as np
import pandas as pd
import scipy as spy
import matplotlib.pyplot as plt


# 1. Calcular el desplazamiento de un objeto por integración numérica.

> a) Escriba, en pseudocódigo, el procedimiento para implementar alguna de las
reglas integrales de Newton-Cotes sobre una función general f(x).

> b) Programe el código respectivo en Python y utilícelo para encontrar el desplazamiento de un móvil, que se mueve en línea recta, entre en el intervalo
de tiempo [0 s, 100 s]. Considere que el móvil se mueve con aceleración
constante, que en t = 0.0 s lleva una rapidez de 0.5 m/s y que en t = 100.0 s
su rapidez es 1.0 m/s.

> c) Defina cinco juegos diferentes de parámetros para el cálculo numérico y en
una tabla muestre estos parámetros y los resultados que generan. Finalmente, identifique los parámetros del cálculo numérico de tal manera que
la incertidumbre del valor reportado sea menor a un 1 % del valor exacto.

## Pseudocódigo 

Se desea implementar la regla del trapecio como método de análisis numérico (regla integral de Newton-Cotes) que permita aproximar el valor de una integral de la forma:

$$\int_{a}^{b}f(x)dx \approx\sum_{k=1}^{N}\frac{f(x_{k-1})+f(x_{k})}{2}\Delta x_{k}$$

Específicamente lo que se busca implementar es:

$$x(100s)=\int_{0}^{100s}v(t)dt=\int_{0}^{100s}(at+v_{0})dt\approx\sum_{k=1}^{N}\frac{v(t_{k-1})+v(t_{k})}{2}\Delta t_{k}$$
Para esto es necesario:

> Definir una función que permita evaluar $v(t_{k})$, esta función recivirá las condicones iniciales del problema y devolvera el valor de $v$ evaluado en un $t_{k}$
> Luego se define una función que aplique el método del trapecio donde se recive como parámetros: las condiciones iniciales del problema, la longitud de los intervalos de tiempo y el valor verdadero de la distancia. Dentro de esta función se llevará acabo lo siguiente:

>> Generar un array unidimensional con valores equiespaciados que empiece en $0s$, termine en $100s$ y tenga $h$ de separación entre valores.

>> Con estos valores se genera una lista donde se guarden los puntos $(t,v(t))$, donde $v(t)$ se obtiene por la función previamenten definida.

>> Implementar el método del trapecio con los datos guardados en la lista

>> Calcular el error dado un valor verdadero.

> Por último se devuelve el valor aproximado de la distancia y el error.

Como segunda parte del código evaluan 5 valores de $N$ para ingresarlos en un pandas DataFrame y generar así una tabla.




In [107]:
def Velocidad(t,t_inicial,t_final,v_inicial,v_final):
  """
  Esta función da la velocidad para un tiempo t dada las condiciones iniciales
  """
  deltaVel = v_final-v_inicial
  deltaT = t_final-t_inicial 
  aceleracion = deltaVel/deltaT
  vel_t = aceleracion*t + v_inicial
  return vel_t  

In [108]:
def AproximacionTrapecio(t,h,t_inicial,t_final,v_inicial,v_final,valorVerdadero):
  """
  Esta función da el valor aproximado y el error del método del
  trapecio cuando se utiliza para averiguar la distancia recorrida por un objeto
  en MRUA cuando se especifícan las condiciones iniciales
  """
  listaDatosTrapecio = []
  tiempo = np.arange(t_inicial,t_final,h)

  for i_t in range(len(tiempo)):
    listaDatosTrapecio.append([tiempo[i_t],Velocidad(tiempo[i_t],t_inicial,t_final,v_inicial,v_final)])
  arrayDatosTrapecio = np.array(listaDatosTrapecio)

  area = 0
  for i_dato in range(1,len(arrayDatosTrapecio)):
    dt = arrayDatosTrapecio[i_dato,0]-arrayDatosTrapecio[i_dato-1,0]
    v_prom = (arrayDatosTrapecio[i_dato,1]+arrayDatosTrapecio[i_dato-1,1])/2
    area += dt*v_prom
  diferencia = valorVerdadero-area
  errorPorcentual = (diferencia/valorVerdadero)*100
  return area, errorPorcentual



In [109]:
a1 = AproximacionTrapecio(100,2,0,100,0.5,1,75)
a2 = AproximacionTrapecio(100,1,0,100,0.5,1,75)
a3 = AproximacionTrapecio(100,0.1,0,100,0.5,1,75)
a4 = AproximacionTrapecio(100,0.01,0,100,0.5,1,75)
a5 = AproximacionTrapecio(100,0.001,0,100,0.5,1,75)
tabla = [[a1[0],a1[1]],[a2[0],a2[1]],[a3[0],a3[1]],[a4[0],a4[1]],[a5[0],a5[1]]]
pd.DataFrame(tabla,index=["h = 2","h = 1","h = 0.1","h = 0.01","h = 0.001"], columns=["Distancia [m]", "%error"])

,Distancia [m],%error
h = 2,73.010000,2.653333
h = 1,74.002500,1.330000
h = 0.1,74.900025,0.133300
h = 0.01,74.990000,0.013333
h = 0.001,74.999000,0.001333


# Discución de resultados:

Es importanten empezar con el hecho de que dada las condiciones iniciales la ecuación que describe el desplazamiento es:

$$x(t)=2.5\times10^{-3}t^{2}+0.5t$$

Por lo que se sabe que el valor verdadero de $v(100s)=75$m. Esto nos lleva a identificar que la principal dificualtad del código fue el proceso de transcribir las ideas "físicas" a un algoritmo que permitiera optener los resultados deseados. Una vez que se tenía en claro como expresar la física en algoritmo el proceso se facilitó ya que pasó a un plano de menor abstracción.

Como se puede ver de la tabla dada por el pandas DataFrame el porcentaje de error se reduce a menos del 1% con pasar de un step de 1 a 0.1. Estos cálculos se realizan en menos de un segundo por lo que se puede alcanzar una muy buena exactitud con el método planteado sin preocuparse por los tiempos de computación.

Por otro lado cabe mencionar Numpy tiene una función llamada trapz (usualmente utilizada como np.trapz()) la cúal implementa el método del trapezoide por lo cual se recomienda el uso de la misma. 



# 2. Determinar el momento en que un objeto pasa por un punto.
> a) Escriba, en pseudocódigo, el procedimiento para implementar el método de
Newton-Raphson para encontrar la raíz de una función general f(x).

> b) Programe el código respectivo en Python y utilícelo para encontrar el valor
del tiempo en que un vehículo que se mueve en línea recta pasa por la posición x = 0.00 m. Utilice que en t = 0.00 s el vehículo se encuentra en x = -5.00
m, parte desde el reposo y se mueve con una aceleración de 0.01 m/s2

> c) Escoja los parámetros del cálculo numérico de tal manera que la incertidumbre del valor reportado sea menor a un 1 % del valor exacto.

# Pseudocódigo

Se desea implementar el método de Newton-Raphson para el cálculo de una raíz. En específico lo que se desea en este caso particular es encotrar la raíz positiva de la siguiente función:

$$x(t)=\frac{a}{2}t^{2}+x_{0}$$

Por medio del siguinte proceso (Newton-Raphson):

$$x_{n+1}=x_{n}-\frac{f(x_{n})}{f^{'}(x_{n})}$$

Para implementar este proceso se requiere:

1. Generar un array que tenga los coeficientes del polinómio.
2. Generar un array que contenga los coeficientes del polinómio derivado.
3. Iterar N veces implementado la ecuación anterior, por medio del valor inicial y los coeficientes polinómicos encontrados .
4. Evaluar el error.
5. Mostrar resultados.

In [39]:
def NewtonRaphson(N,x_ini,a,posicionInicial):
  """
  Esta función calcula las raíces dada una función polinómica por medio del
  método Newton Raphson con N iteraciones. 
  """
  polinomio = np.poly1d([a/2,0,posicionInicial])
  derivadaPol = np.polyder(polinomio)
  raizP = polinomio.r[1]

  x_k = x_ini
  for k in range(N):
    x_k = x_k - polinomio(x_k)/derivadaPol(x_k)
  error = (abs(x_k-raizP)/raizP) *100
  return x_k,x_ini,error

In [50]:

a1 = NewtonRaphson(5,5,0.01,-5)
a2 = NewtonRaphson(5,10,0.01,-5)
a3 = NewtonRaphson(5,15,0.01,-5)
a4 = NewtonRaphson(5,20,0.01,-5)
a5 = NewtonRaphson(5,60,0.01,-5)
tabla = [[a1[0],a1[1],a1[2]],[a2[0],a2[1],a2[2]],[a3[0],a3[1],a3[2]],[a4[0],a4[1],a4[2]],[a5[0],a5[1],a5[2]]]
pd.DataFrame(tabla,index=["N = 5","N = 5","N = 5","N = 5","N = 5"], columns=["Distancia [m]","Valor Inicial" ,"%error"])

,Distancia [m],Valor Inicial,%error
N = 5,31.625116,5,7.397971e-03
N = 5,31.622777,10,1.583446e-07
N = 5,31.622777,15,9.324774e-13
N = 5,31.622777,20,0.000000e+00
N = 5,31.622777,60,1.123467e-14


# Discución de resultados:

En este código las complicaciones principales se dan a la hora de determinar la forma en la que se evalua $f$ y $f’$, en este caso con el uso de numpy se simplifica el proceso sin embargo se puede ver la necesidad de implementar diferencias centradas etc. 

Otro problema es que el método converge de forma distinta dependiendo de la cantidad de iteraciones y el valor inicial, por lo que seleccionar valores iniciales cercanos a la raíz es óptimo. Una vez que se seleccionan valores cercanos a la ráiz el método converge rápido por lo que alcanzar la precisión deseada es sensillo.

Por último se puede ver que si se utiliza numpy.poly1d().r se puede encontrar las raíces de un polinómio dado los coeficientes del mismo.
